# 🇲🇳 Монгол хэл дээр Sentiment Analysis Transformer
Энэхүү notebook-д Трансформер архитектур ашиглан  sentiment ангиллын загварыг PyTorch дээр хэрэгжүүлнэ.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1101RLr7YC2M-leFtDIyVu6BXX1GEUVzS?usp=sharing)

In [ ]:
# 📦 Сангуудыг импортлох
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# 📚 Жишээ өгөгдөл — 20 өгүүлбэр
data = [('Энэ кино үнэхээр гайхалтай байлаа', 1), ('Би энэ номонд маш их дуртай', 1), ('Үнэхээр маш ойлгомжтой, үр дүнтэй байсан', 1), ('Энэ ресторан үнэхээр амтгүй хоолтой', 0), ('Үйлчилгээ маш муу байлаа', 0), ('Киноны үйл явдал уйтгартай байсан', 0), ('Энэ газар надад маш их таалагдсан', 1), ('Би дахиж хэзээ ч энэ дэлгүүрт орохгүй', 0), ('Гайхамшигтай байгальтай газар байна', 1), ('Багшийн тайлбар маш тодорхой, сайн байсан', 1), ('Үнэтэй мөртлөө чанар муутай бүтээгдэхүүн', 0), ('Би энэ аппликейшнийг өдөр бүр ашигладаг', 1), ('Энэ кино миний цагийг дэмий үрсэн', 0), ('Би энэ газрын хоолонд дуртай', 1), ('Хэтэрхий удаан хүргэлттэй', 0), ('Маш сайн бичигдсэн нийтлэл байна', 1), ('Тоглоом байнга алдаа өгдөг', 0), ('Гоё дизайнтай, хэрэглэхэд хялбар', 1), ('Энэ үйлчилгээ үнэхээр сэтгэл гонсойлгосон', 0), ('Номын агуулга маш сонирхолтой байсан', 1)]
texts, labels = zip(*data)

In [ ]:
# 🔤 Tokenization + Vocabulary үүсгэх
from collections import defaultdict
import re
def tokenize(text):
    return re.findall(r'\w+', text.lower())

word2idx = {'<PAD>': 0, '<UNK>': 1}
for text in texts:
    for token in tokenize(text):
        if token not in word2idx:
            word2idx[token] = len(word2idx)
idx2word = {i: w for w, i in word2idx.items()}
print('idx2word :',idx2word)
vocab_size = len(word2idx)
print('vocab_size :',vocab_size)

idx2word : {0: '<PAD>', 1: '<UNK>', 2: 'энэ', 3: 'кино', 4: 'үнэхээр', 5: 'гайхалтай', 6: 'байлаа', 7: 'би', 8: 'номонд', 9: 'маш', 10: 'их', 11: 'дуртай', 12: 'сургалт', 13: 'ойлгомжтой', 14: 'үр', 15: 'дүнтэй', 16: 'байсан', 17: 'ресторан', 18: 'амтгүй', 19: 'хоолтой', 20: 'үйлчилгээ', 21: 'муу', 22: 'киноны', 23: 'үйл', 24: 'явдал', 25: 'уйтгартай', 26: 'газар', 27: 'надад', 28: 'таалагдсан', 29: 'дахиж', 30: 'хэзээ', 31: 'ч', 32: 'дэлгүүрт', 33: 'орохгүй', 34: 'гайхамшигтай', 35: 'байгальтай', 36: 'байна', 37: 'багшийн', 38: 'тайлбар', 39: 'тодорхой', 40: 'сайн', 41: 'үнэтэй', 42: 'мөртлөө', 43: 'чанар', 44: 'муутай', 45: 'бүтээгдэхүүн', 46: 'аппликейшнийг', 47: 'өдөр', 48: 'бүр', 49: 'ашигладаг', 50: 'миний', 51: 'цагийг', 52: 'дэмий', 53: 'үрсэн', 54: 'газрын', 55: 'хоолонд', 56: 'хэтэрхий', 57: 'удаан', 58: 'хүргэлттэй', 59: 'бичигдсэн', 60: 'нийтлэл', 61: 'тоглоом', 62: 'байнга', 63: 'алдаа', 64: 'өгдөг', 65: 'гоё', 66: 'дизайнтай', 67: 'хэрэглэхэд', 68: 'хялбар', 69: 'сэтгэл',

In [ ]:
# 🧾 Text-ийг индекс болгох
max_len = 10
def encode(text):
    tokens = tokenize(text)
    ids = [word2idx.get(t, 1) for t in tokens]
    return ids[:max_len] + [0] * (max_len - len(ids))

X = torch.tensor([encode(t) for t in texts])
y = torch.tensor(labels)

In [ ]:
# 📦 Dataset & DataLoader
class SentimentDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

train_ds = SentimentDataset(X, y)
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)

In [ ]:
# 🧠 Трансформер загвар
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, d_model=64, nhead=4, num_layers=2, num_classes=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = nn.Parameter(torch.randn(1, max_len, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x) + self.positional_encoding[:, :x.size(1), :]
        x = self.transformer(x)
        x = x.mean(dim=1)  # pooling
        return self.classifier(x)

model = TransformerClassifier(vocab_size)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
# ⚙️ Алдагдлын функц ба optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 📊 Сургалтын ба тестийн өгөгдөлд хуваах
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
train_ds = SentimentDataset(X_train, y_train)
test_ds = SentimentDataset(X_test, y_test)
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=4)

In [ ]:
# 🧪 Үнэлгээ хийх функц
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for xb, yb in loader:
            preds = model(xb)
            predicted = torch.argmax(preds, dim=1)
            correct += (predicted == yb).sum().item()
            total += yb.size(0)
    acc = correct / total
    return acc

In [ ]:
# 🏋️‍♂️ Сургалт + үнэлгээ
for epoch in range(100):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        preds = model(xb)
        loss = criterion(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    train_acc = evaluate(model, train_loader)
    test_acc = evaluate(model, test_loader)
    print(f"Epoch {epoch}: Loss = {total_loss:.4f} | Train Acc = {train_acc:.2f}, Test Acc = {test_acc:.2f}")

Epoch 0: Loss = 0.1716 | Train Acc = 1.00, Test Acc = 0.67
Epoch 1: Loss = 0.1399 | Train Acc = 1.00, Test Acc = 0.67
Epoch 2: Loss = 0.1385 | Train Acc = 1.00, Test Acc = 0.83
Epoch 3: Loss = 0.1215 | Train Acc = 1.00, Test Acc = 0.83
Epoch 4: Loss = 0.1088 | Train Acc = 1.00, Test Acc = 0.83
Epoch 5: Loss = 0.1164 | Train Acc = 1.00, Test Acc = 0.83
Epoch 6: Loss = 0.1229 | Train Acc = 1.00, Test Acc = 0.83
Epoch 7: Loss = 0.0924 | Train Acc = 1.00, Test Acc = 0.67
Epoch 8: Loss = 0.0899 | Train Acc = 1.00, Test Acc = 0.67
Epoch 9: Loss = 0.0938 | Train Acc = 1.00, Test Acc = 0.67
Epoch 10: Loss = 0.0854 | Train Acc = 1.00, Test Acc = 0.67
Epoch 11: Loss = 0.0841 | Train Acc = 1.00, Test Acc = 0.67
Epoch 12: Loss = 0.0905 | Train Acc = 1.00, Test Acc = 0.67
Epoch 13: Loss = 0.0718 | Train Acc = 1.00, Test Acc = 0.67
Epoch 14: Loss = 0.0916 | Train Acc = 1.00, Test Acc = 0.67
Epoch 15: Loss = 0.0708 | Train Acc = 1.00, Test Acc = 0.67
Epoch 16: Loss = 0.0900 | Train Acc = 1.00, Test A

In [ ]:
# 🔮 Inference — өгүүлбэр таамаглах функц
def predict_sentiment(text, model):
    model.eval()
    with torch.no_grad():
        encoded = torch.tensor([encode(text)])  # өгүүлбэрийг encode хийнэ
        logits = model(encoded)                 # загвар дээр дамжуулна
        prediction = torch.argmax(logits, dim=1).item()
        return "Эерэг" if prediction == 1 else "Сөрөг"

# 🧪 Жишээ өгүүлбэрүүд дээр test хийх:
test_sentences = [
    "Энэ кино үнэхээр гоё байлаа",
    "Үйлчилгээ маш муу байсан",
    "Би энэ аппликейшнийг өдөр бүр хэрэглэдэг",
    "Хоол нь амт муутай байлаа"
]

for sentence in test_sentences:
    result = predict_sentiment(sentence, model)
    print(f"'{sentence}' → {result}")


'Энэ кино үнэхээр гоё байлаа' → Сөрөг
'Үйлчилгээ маш муу байсан' → Сөрөг
'Би энэ аппликейшнийг өдөр бүр хэрэглэдэг' → Эерэг
'Хоол нь амт муутай байлаа' → Сөрөг
